# 여기서부터 진짜입니다#######################

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, accuracy_score, r2_score

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/3차-따릉이/data/data_요일_sum.csv')
data

,금,목,수,월,일,토,화,자치구,시각,기온,풍속,강수량,미세먼지농도,역,이용건수,달
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,-5.1,1.0,0.0,27.0,0,8,1
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,-5.1,1.0,0.0,27.0,1,16,1
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1,-5.0,0.8,0.0,20.0,0,14,1
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1,-5.0,0.8,0.0,20.0,1,15,1
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,2,-4.8,1.3,0.0,21.0,0,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424916,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11,21,-6.2,1.4,0.0,42.0,1,12,12
424917,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11,22,-5.6,1.6,0.0,42.0,0,4,12
424918,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11,22,-5.6,1.6,0.0,42.0,1,3,12
424919,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11,23,-6.3,1.4,0.0,43.0,0,1,12


In [ ]:
features = data.drop(['이용건수'], axis=1)
target = data['이용건수']

In [ ]:
scaler = MinMaxScaler()
feature_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_scaled, target, random_state=0)

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

randomforest = RandomForestRegressor()
# parameter 넣어줄 값들 dict 형태로 정의해주기
param = {
         }

gird_randomforest = RandomizedSearchCV(randomforest, scoring='r2', param_distributions=param, cv=kfold, return_train_score=True, verbose=2)

# GridSearch 하면서 모든 파라미터값들에 대해 학습 수행
gird_randomforest.fit(X_train, y_train)

# 각 파라미터값들에 대한 모델 결과값들이 cv_results_ 객체에 할당됨
scores_df = pd.DataFrame(gird_randomforest.cv_results_)

# score 결과값(ndarray형태로 할당됨) 중 특정 칼럼들만 가져오기 
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ................................................. , total= 2.3min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV] ................................................. , total= 2.4min
[CV]  ................................................................
[CV] ................................................. , total= 2.4min
[CV]  ................................................................
[CV] ................................................. , total= 2.3min
[CV]  ................................................................
[CV] ................................................. , total= 2.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 13.1min finished


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,{},0.888693,1,0.89288,0.888574,0.888407


In [ ]:
 # 최적의 파라미터는 best_params_에 할당되어있음
print(f"최적의 파라미터 : {gird_randomforest.best_params_}")
print(f"최적의 파라미터로 모델의 r2 : {gird_randomforest.best_score_}")

print()
print()

# 최적의 파라미터로 학습되어 있는 모델링 할당
estimator = gird_randomforest.best_estimator_

# 최적의 모델로 예측해보고 실제값이랑 정확도 비교
y_pred = estimator.predict(X_test)
print(f"실제값과 예측값 r2 score : {r2_score(y_test, y_pred)}")

최적의 파라미터 : {}
최적의 파라미터로 모델의 r2 : 0.8886925148235678


실제값과 예측값 r2 score : 0.8925682523080453
